# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## **Question 1** ##
### What test is appropriate for this problem? Does CLT apply? ###

Generally the z-statistic is used if the sample size is over 30 and you know the population standard deviation. The t-statistic is used if you do not know the population standard deviation and the sample size is less than 30. While our sample size is over 30, we do not know the standard deviation for the population, so I would choose the t-statistic. I will use a two-sample t-test because we are looking at a binomial population.

The CLT applies in this case because n is larger than 30 and the distribution of the sample can be assumed to be normal.


## Question 2 ##
### What are the null and alternate hypotheses? ###

Null hypothesis = black ('b') and white ('w') sounding names receive the same number of callbacks.

Alternative hypothesis = black ('b') and white ('w') sounding names do not receive the same number of callbacks.

## Question 3 ##
### Compute margin of error, confidence interval, and p-value ###

In [8]:
#Separate into two datasets by race
w = data[data.race=='w']
b = data[data.race=='b']

#Define the number resumes for each group
n_w = len(w)
n_b = len(b)

#Define the proportion of callbacks for each racial group
prop_w = np.sum(w.call) / len(w)
prop_b = np.sum(b.call) / len(b)
print('Proportion of callbacks for whites: ', prop_w)
print('Proportion of callbacks for blacks: ', prop_b)

#Difference in proportions of callbacks between the racial groups
prop_diff = prop_w - prop_b
print('Difference in proportions of callbacks: ', prop_diff)

#Proportion of callbacks for the whole sample
phat = (np.sum(w.call) + np.sum(b.call)) / (len(w) + len(b))
print('Proportion of callbacks for entire sample: ', phat)

#2-sample t-test
ttest, p = stats.ttest_ind(w.call,b.call)
print('T-test: ', ttest)
print('p-value: ', p)

#Margin of Error
moe = 1.96 * np.sqrt(phat * (1 - phat) * ((1 / n_w) + (1 / n_b)))
print('Margin of Error: ', moe)

#Confidence Interval
ci = prop_diff + np.array([-1, 1]) * moe
print('Confidence Interval: ', ci)

#p-value
pval = stats.norm.cdf(-ttest) * 2
print("P-value: ", pval)

('Proportion of callbacks for whites: ', 0.096509240246406572)
('Proportion of callbacks for blacks: ', 0.064476386036960986)
('Difference in proportions of callbacks: ', 0.032032854209445585)
('Proportion of callbacks for entire sample: ', 0.080492813141683772)
('T-test: ', 4.1147052908617514)
('p-value: ', 3.9408021031288859e-05)
('Margin of Error: ', 0.0)
('Confidence Interval: ', array([ 0.03203285,  0.03203285]))
('P-value: ', 3.876744018936831e-05)


White-sounding names receive callbacks 9.65% of the time, while black-sounding names receive callbacks 6.45% of the time. The p-value is is very small (.0000388), so we can reject the null hypothesis that black ('b') and white ('w') sounding names receive the same number of callbacks.

The difference between callback rates between the racial groups is .032.

## Question 4 ##
### Write a story describing the statistical significance in the context or the original problem ###

Racial discrimination continues to be a very real problem in the United States and around the world. A recent study looked at whether the perceived race of an applicant had an impact on the callback rate for a job application. Resumes were randomly assigned black-sounding or white-sounding names and researchers observed whether or not the difference had an impact on employer requests for interviews. Black-sounding names received requests for interviews 6.5% of the time, while white-sounding names received interview requests 9.7% of the time. We wanted to determine whether the difference between the groups was statistically significant, or whether it was a random result. To make this determination, we used a two-sample t-test. Our analysis shows that the perceived race of a candidate has an effect on the callback success rate.


## Question 5 ##
### Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis? ###

No, our analysis does not mean that race/name is the most important factor in callback success. While the race (or perceived race from the name) does have an effect on callback success, there are other important factors employers take into consideration. For example, we do not know how researchers controlled for education and experience. Were identical resumes given two different names? Or did the resumes have a range of education and experience? To amend my analysis, I would look more into the resumes and how the education, experience, and location compared between the white-sounding name resumes and black-sounding name resumes.